In [9]:
from app.MarketScraper import MarketScraper
from app.MariaDBClient import MariaDBClient
import pandas as pd

In [10]:
market_scraper = MarketScraper()
mariadbclient = MariaDBClient()

In [11]:
supermercados = mariadbclient.get_active_tables()
print(supermercados)

['atomo', 'blowmax', 'modomarket', 'segal']


In [12]:
total_items = []
total_stats = []
for supermercado in supermercados:
    data = market_scraper.fetch_data(supermercado)
    items = market_scraper.get_items(data)
    total_items.append(items)
    stas = market_scraper.get_stats(data)
    total_stats.append(stas)

total_items_df = pd.concat(total_items, ignore_index=True)
total_stats_df = pd.concat(total_stats, ignore_index=True)



In [13]:
display(total_items_df)

,__ingestion_timestamp,nombre_crudo,nombre,marca,descripcion,volumen,precio,id_supermercado,url
0,2025-04-12 22:25:51.807713-03:00,SHAMPOO HEAD & SHOULDER ANTI COMEZON 375 ML,HEAD & SHOULDERSOULDERS ANTI COMEZON 375,HEAD & SHOULDERSOULDERS,ANTI COMEZON,375,7875.0,1,https://atomoconviene.com/atomo-ecommerce/sham...
1,2025-04-12 22:25:51.807713-03:00,SHAMPOO FLIAR SUAVE BOMBA DELICIOSA 930 ML.,SUAVE BOMBA DELICIOSA 930,SUAVE,BOMBA DELICIOSA,930,3009.0,1,https://atomoconviene.com/atomo-ecommerce/sham...
2,2025-04-12 22:25:51.807713-03:00,SHAMPOO HEAD & SHOULDER ANTI COMEZON 180 ML,HEAD & SHOULDERSOULDERS ANTI COMEZON 180,HEAD & SHOULDERSOULDERS,ANTI COMEZON,180,4703.0,1,https://atomoconviene.com/atomo-ecommerce/sham...
3,2025-04-12 22:25:51.807713-03:00,SHAMPOO PANTENE KERATINA 400 ML.,PANTENE KERATINA 400,PANTENE,KERATINA,400,6524.0,1,https://atomoconviene.com/atomo-ecommerce/sham...
4,2025-04-12 22:25:51.807713-03:00,SHAMPOO PANTENE DETOX 200 ML.,PANTENE DETOX 200,PANTENE,DETOX,200,2596.1,1,https://atomoconviene.com/atomo-ecommerce/sham...
...,...,...,...,...,...,...,...,...,...
1147,2025-04-12 22:30:29.953454-03:00,TRESEMMÉ SHAMPOO 400ml BAJO POO NUTRICIÓN,TRESEMME BAJO POO NUTRICION 400,TRESEMME,BAJO POO NUTRICION,400,4569.0,4,https://www.casa-segal.com/producto/tresemme-s...
1148,2025-04-12 22:30:29.953454-03:00,TRESEMMÉ SHAMPOO 400ml DETOX CAPILAR,TRESEMME DETOX CAPILAR 400,TRESEMME,DETOX CAPILAR,400,4569.0,4,https://www.casa-segal.com/producto/tresemme-s...
1149,2025-04-12 22:30:29.953454-03:00,TRESEMMÉ SHAMPOO 400ml HIDRATACIÓN PROFUNDA,TRESEMME HIDRATACION PROFUNDA 400,TRESEMME,HIDRATACION PROFUNDA,400,4569.0,4,https://www.casa-segal.com/producto/tresemme-s...
1150,2025-04-12 22:30:29.953454-03:00,TRESEMMÉ SHAMPOO 400ml REGENERACIÓN TRESPLEX,TRESEMME REGENERACION TRESPLEX 400,TRESEMME,REGENERACION TRESPLEX,400,4569.0,4,https://www.casa-segal.com/producto/tresemme-s...


In [14]:
display(total_items_df)

,__ingestion_timestamp,nombre_crudo,nombre,marca,descripcion,volumen,precio,id_supermercado,url
0,2025-04-12 22:25:51.807713-03:00,SHAMPOO HEAD & SHOULDER ANTI COMEZON 375 ML,HEAD & SHOULDERSOULDERS ANTI COMEZON 375,HEAD & SHOULDERSOULDERS,ANTI COMEZON,375,7875.0,1,https://atomoconviene.com/atomo-ecommerce/sham...
1,2025-04-12 22:25:51.807713-03:00,SHAMPOO FLIAR SUAVE BOMBA DELICIOSA 930 ML.,SUAVE BOMBA DELICIOSA 930,SUAVE,BOMBA DELICIOSA,930,3009.0,1,https://atomoconviene.com/atomo-ecommerce/sham...
2,2025-04-12 22:25:51.807713-03:00,SHAMPOO HEAD & SHOULDER ANTI COMEZON 180 ML,HEAD & SHOULDERSOULDERS ANTI COMEZON 180,HEAD & SHOULDERSOULDERS,ANTI COMEZON,180,4703.0,1,https://atomoconviene.com/atomo-ecommerce/sham...
3,2025-04-12 22:25:51.807713-03:00,SHAMPOO PANTENE KERATINA 400 ML.,PANTENE KERATINA 400,PANTENE,KERATINA,400,6524.0,1,https://atomoconviene.com/atomo-ecommerce/sham...
4,2025-04-12 22:25:51.807713-03:00,SHAMPOO PANTENE DETOX 200 ML.,PANTENE DETOX 200,PANTENE,DETOX,200,2596.1,1,https://atomoconviene.com/atomo-ecommerce/sham...
...,...,...,...,...,...,...,...,...,...
1147,2025-04-12 22:30:29.953454-03:00,TRESEMMÉ SHAMPOO 400ml BAJO POO NUTRICIÓN,TRESEMME BAJO POO NUTRICION 400,TRESEMME,BAJO POO NUTRICION,400,4569.0,4,https://www.casa-segal.com/producto/tresemme-s...
1148,2025-04-12 22:30:29.953454-03:00,TRESEMMÉ SHAMPOO 400ml DETOX CAPILAR,TRESEMME DETOX CAPILAR 400,TRESEMME,DETOX CAPILAR,400,4569.0,4,https://www.casa-segal.com/producto/tresemme-s...
1149,2025-04-12 22:30:29.953454-03:00,TRESEMMÉ SHAMPOO 400ml HIDRATACIÓN PROFUNDA,TRESEMME HIDRATACION PROFUNDA 400,TRESEMME,HIDRATACION PROFUNDA,400,4569.0,4,https://www.casa-segal.com/producto/tresemme-s...
1150,2025-04-12 22:30:29.953454-03:00,TRESEMMÉ SHAMPOO 400ml REGENERACIÓN TRESPLEX,TRESEMME REGENERACION TRESPLEX 400,TRESEMME,REGENERACION TRESPLEX,400,4569.0,4,https://www.casa-segal.com/producto/tresemme-s...


In [17]:
mariadbclient.insert_into_productos(total_items_df)
mariadbclient.insert_into_db("webscraping_info", total_stats_df)
mariadbclient.insert_into_db("historico", total_items_df)